In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\ramak\\OneDrive\\Desktop\\P2\\CatDogClassification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    learning_rate: float
    classes: int
    input_shape: list
    include_top: bool
    weights: str

In [4]:
from src.CatDogClassification.constants import *
from src.CatDogClassification.utils.common import read_yaml, create_directory

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath = Config_File_Path, param_filepath = Params_File_Path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)

        create_directory([self.config.main_dir])

    def get_prepared_base_model(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directory([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = config.root_dir,
            base_model_path = config.base_model_path,
            updated_base_model_path = config.updated_base_model_path,
            learning_rate = self.params.LEARNING_RATE,
            classes = self.params.CLASSES,
            input_shape = self.params.INPUT_SHAPE,
            include_top = self.params.INCLUDE_TOP,
            weights = self.params.WEIGHTS,
        )

        return prepare_base_model_config


In [6]:
import os
from src.CatDogClassification import logger
import tensorflow as tf
from tensorflow import keras
from keras import layers, models


[2023-12-15 14:55:47,949: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.model = models.Sequential()
        self.input_shape = self.config.input_shape


    @staticmethod
    def prepare_base_model(model,input_shape,weights,include_top,learning_rate):
        
        base_model = tf.keras.applications.VGG16(weights=weights, include_top=include_top, input_shape=input_shape)
        base_model.trainable = False

        model = models.Sequential([
            base_model,
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ])


        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=learning_rate,
            decay_steps=10000,
            decay_rate=0.9)

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
            loss='binary_crossentropy',
            metrics=['accuracy'])

        model.summary()

        return model
    
    def updated_base_model(self):
        self.model = self.prepare_base_model(
            model = self.model,
            input_shape = self.input_shape,
            weights = self.config.weights,
            include_top = self.config.include_top,
            learning_rate = self.config.learning_rate
        )

        self.save_model(path = self.config.updated_base_model_path, model = self.model)

    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        model.save(path)
        logger.info("Saving model")


In [8]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepared_base_model()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    # prepare_base_model.base_model()
    prepare_base_model.updated_base_model()
except Exception as e:
    raise e

[2023-12-15 14:55:50,450: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\CatDogClassification\config\config.yaml loaded]
[2023-12-15 14:55:50,469: INFO: common: yaml file: C:\Users\ramak\OneDrive\Desktop\P2\CatDogClassification\params.yaml loaded]
[2023-12-15 14:55:50,471: INFO: common: Created directory at: artifacts]
[2023-12-15 14:55:50,474: INFO: common: Created directory at: artifacts/prepare_base_model]


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 512)               4194816   
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 18910017 (72.14 MB)
Trainable params: 4195329 (16.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


C:\Users\ramak\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[2023-12-15 14:55:54,118: INFO: 1218904505: Saving model]
